In [1]:
import src.models.builders as mb
import src.models.helpers as mh
import src.inference.helpers as ih
import src.modelling_o2.fit_ho2sat_ar as fit_ho2sat_ar

import numpy as np

In [2]:
height = 165
age = 26
sex = "Male"
hfev1_prior = {"type": "default", "height": height, "age": age, "sex": sex}
ho2sat_prior = {
    "type": "default",
    "height": height,
    "sex": sex,
}
(
    model,
    inf_alg,
    HFEV1,
    FEV1,
    AR,
    # model,
    # inf_alg,
    # HFEV1,
    # FEV1,
    # HO2Sat,
    # O2SatFFA,
    # AR,
) = mb.build_FEV1_O2_point_in_time_model(hfev1_prior, ho2sat_prior)

*** Building FEV1 and O2 point in time model ***
Time to build variables: 0.005990266799926758
+---------------------------+-----------+
| Airway Resistance (%)(0)  | 0.0111111 |
+---------------------------+-----------+
| Airway Resistance (%)(1)  | 0.0111111 |
+---------------------------+-----------+
| Airway Resistance (%)(2)  | 0.0111111 |
+---------------------------+-----------+
| Airway Resistance (%)(3)  | 0.0111111 |
+---------------------------+-----------+
| Airway Resistance (%)(4)  | 0.0111111 |
+---------------------------+-----------+
| Airway Resistance (%)(5)  | 0.0111111 |
+---------------------------+-----------+
| Airway Resistance (%)(6)  | 0.0111111 |
+---------------------------+-----------+
| Airway Resistance (%)(7)  | 0.0111111 |
+---------------------------+-----------+
| Airway Resistance (%)(8)  | 0.0111111 |
+---------------------------+-----------+
| Airway Resistance (%)(9)  | 0.0111111 |
+---------------------------+-----------+
| Airway Resistance (%)

/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:313: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  val, abserr = integrate.quad(


KeyboardInterrupt: 

In [2]:
height = 165
age = 26
sex = "Male"
hfev1_prior = {"type": "default", "height": height, "age": age, "sex": sex}
ho2sat_prior = {
    "type": "default",
    "height": height,
    "sex": sex,
}

In [5]:
import time

import numpy as np
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import BeliefPropagation
from pgmpy.models import BayesianNetwork

import src.modelling_fev1.pred_fev1 as pred_fev1
import src.modelling_o2.o2satffa_factor as o2satffa_factor
import src.models.helpers as mh

# Compute CPTs
HFEV1 = mh.variableNode("Healthy FEV1 (L)", 1, 6, 0.1, prior=hfev1_prior)
AR = mh.variableNode("Airway Resistance (%)", 0, 90, 2, prior={"type": "uniform"})
ecFEV1 = mh.variableNode("FEV1 (L)", 0, 6, 0.1, prior=None)
ecFEV1.prior = cpt_ecfev1

HO2Sat = mh.variableNode(
    "Healthy O2 Saturation (%)", 90, 100, 1, prior=ho2sat_prior
)
O2SatFFA = mh.variableNode(
    "O2 Sat if fully functional alveoli (%)", 70, 100, 1, prior=None
)

# Encode nodeVariables
HFEV1 = mh.encode_node_variable(HFEV1)
AR = mh.encode_node_variable(AR)
ecFEV1 = mh.encode_node_variable(ecFEV1)

# Decode nodeVariables
HFEV1 = mh.decode_node_variable(HFEV1)
AR = mh.decode_node_variable(AR)
ecFEV1 = mh.decode_node_variable(ecFEV1)

# Build model with PGMPY
model = BayesianNetwork(
    [
        (HFEV1.name, ecFEV1.name),
        (AR.name, ecFEV1.name),
        # (HO2Sat.name, O2SatFFA.name),
        # (AR.name, O2SatFFA.name),
    ]
)

prior_hfev1 = TabularCPD(
    variable=HFEV1.name,
    variable_card=len(HFEV1.bins),
    values=HFEV1.prior,
    evidence=[],
    evidence_card=[],
)
# prior_ho2sat = TabularCPD(
#     variable=HO2Sat.name,
#     variable_card=len(HO2Sat.bins),
#     values=HO2Sat.prior,
#     evidence=[],
#     evidence_card=[],
# )
prior_ar = TabularCPD(
    variable=AR.name,
    variable_card=len(AR.bins),
    values=AR.prior,
    evidence=[],
    evidence_card=[],
)
cpt_fev1 = TabularCPD(
    variable=ecFEV1.name,
    variable_card=len(ecFEV1.bins),
    values=ecFEV1.prior,
    evidence=[HFEV1.name, AR.name],
    evidence_card=[len(HFEV1.bins), len(AR.bins)],
)
# cpt_o2_sat_ffa = TabularCPD(
#     variable=O2SatFFA.name,
#     variable_card=len(O2SatFFA.bins),
#     values=o2satffa_factor.calc_cpt_O2SatFFA_HO2Sat_AR(
#         O2SatFFA, HO2Sat, AR, debug=False
#     ),
#     evidence=[HO2Sat.name, AR.name],
#     evidence_card=[len(HO2Sat.bins), len(AR.bins)],
# )

model.add_cpds(cpt_fev1, prior_ar, prior_hfev1)
# model.add_cpds(cpt_fev1, prior_ar, prior_hfev1, prior_ho2sat, cpt_o2_sat_ffa)

model.check_model()
inf_alg = BeliefPropagation(model)

Defining gaussian prior with mu = 97.33, sigma = 1.0304


/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_15656/1947004663.py:45: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prior_hfev1 = TabularCPD(
/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_15656/1947004663.py:59: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  prior_ar = TabularCPD(
/var/folders/zq/v2r6yn111s3gpdf8lzf72xvw0000gn/T/ipykernel_15656/1947004663.py:66: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  cpt_fev1 = TabularCPD(


In [4]:
cpt_ecfev1 = mh.calc_pgmpy_cpt_X_x_1_minus_Y(HFEV1, AR, ecFEV1)

/Users/tristan.trebaol/Desktop/PhD/Code/phd/src/models/helpers.py:312: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  val, abserr = integrate.quad(
